In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Take aways
* engineer features to model the major time series components (trends, seasons, and cycles),
* visualize time series with many kinds of time series plots,
* create forecasting hybrids that combine the strengths of complementary models, and
* adapt machine learning methods to a variety of forecasting tasks.

## 1. Forecasting

In [ ]:
%matplotlib inline
from warnings import simplefilter
simplefilter("ignore")
import matplotlib.pyplot as plt
import seaborn as sns

# load books data

books = pd.read_csv('../input/ts-course-data/book_sales.csv',
                   index_col='Date', parse_dates=['Date']).drop('Paperback', axis=1)
books.head()

### Linear Regression with Time Series

Linear Regression algorithm learns how to make a weighted sum from its input features. For two features, we would have:
`target = weigth_1 * feature_1 + weight_2 * feature_2  + bias`
weights (coefficients), bias (intercept)


### Time step features, 
are features we can derive directly from the time index.

In [ ]:
books['Time'] = np.arange(len(books.index))
books.head()

In [ ]:
plt.rc(
    "figure",
    autolayout=True,
    figsize=(11, 4),
    titlesize=18,
    titleweight='bold',
)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
%config InlineBackend.figure_format = 'retina'

# Linear regression with the time dummy produces the model
# target = weigth * time + bias


fig, ax = plt.subplots()
ax.plot("Time", "Hardcover", data=books, color='0.75')
ax = sns.regplot(data=books,x='Time', y='Hardcover',ci=None, scatter_kws=dict(color='0.25'))
ax.set_title('Time Plot of Hardcover Sales');

### Lag features
To make a lag feature we shift the observations of the target series so that they appear to have occured later in time. Here we've created a 1-step lag feature, though shifting by multiple steps is possible too

In [ ]:
books['lag_1'] = books['Hardcover'].shift()
books = books.reindex(columns=['Hardcover', 'lag_1'])
books.head()

In [ ]:
# target = weight * lag + bias
fig, ax = plt.subplots()
ax = sns.regplot(data=books, x='lag_1', y='Hardcover',  ci=None,  scatter_kws=dict(color='0.25'))
ax.set_aspect('equal')
ax.set_title('Lag Plot of Hardcover Sales');

You can see from the lag plot that sales on one day (Hardcover) are correlated with sales from the previous day (Lag_1). When you see a relationship like this, you know a lag feature will be useful.

In [ ]:
# tunnel traffic
tunnel = pd.read_csv('../input/ts-course-data/tunnel.csv', index_col='Day', parse_dates=['Day'])
tunnel = tunnel.to_period()
tunnel.head()

In [ ]:
# time-step feature
tunnel['Time'] = np.arange(len(tunnel.index))
tunnel.head()

In [ ]:
from sklearn.linear_model import LinearRegression

# training data
X = tunnel.loc[ : , ['Time']]
y = tunnel.loc[ : , 'NumVehicles']

# train the model 
lrg = LinearRegression().fit(X, y)

y_pred = pd.Series(lrg.predict(X), index=X.index)

In [ ]:
ax = y.plot()
ax = y_pred.plot(ax=ax, linewidth=3)
ax.set_title('Time Plot of Tunnel Traffic');

In [ ]:
# lag feature

tunnel['lag_1'] = tunnel['NumVehicles'].shift(1)
tunnel = tunnel.dropna()
tunnel.head()

In [ ]:
# training data
X = tunnel.loc[ : , ['lag_1']]
y = tunnel.loc[ : , 'NumVehicles']

lrg = LinearRegression().fit(X, y)

y_pred = pd.Series(lrg.predict(X), index=X.index)

In [ ]:
fig, ax = plt.subplots()
ax.plot(X['lag_1'], y, '.', color='0.25')
ax.plot(X['lag_1'], y_pred)
ax.set_aspect('equal')
ax.set_ylabel('NumVehicles')
ax.set_xlabel('Lag_1')
ax.set_title('Lag Plot of Tunnel Traffic');

In [ ]:
ax = y.plot()
ax = y_pred.plot()

## 2. Trend
The trend component of a time series represents a persistent, long-term change in the mean of the series. 

### Moving Average plots
To see what kind of trend a time series might have, we can use a moving average plot

In [ ]:
# engineering trend

# tunnel traffic
tunnel = pd.read_csv('../input/ts-course-data/tunnel.csv', index_col='Day', parse_dates=['Day'])
tunnel = tunnel.to_period()
tunnel.head()

In [ ]:
# moving average plot within a year
# use the rolling method to begin a windowed computation, Follow this by the mean method to compute the average over the window

moving_average = tunnel.rolling(
    window=365,  # 365-day window
    center=True, # puts average at the center of the window
    min_periods=185 #choose abt half the window size
).mean() # compute the mean ( could be median, std, min, max, ...)

ax = tunnel.plot(style='.', color='0.5')
moving_average.plot(
ax=ax, linewidth=3, title="Tunnel Traffic - 365-Day Moving Average", legend=False)


In [ ]:
# dummy time engineering with DeterministicProcess
from statsmodels.tsa.deterministic import DeterministicProcess

dp = DeterministicProcess(
        index=tunnel.index, # dates from the training data
        constant=True,  # dummy feature for the y_intercept
        order=1, # the time dummy trend ( 1 for linear, 2 for quadratic, 3 for cubic )
        drop=True # drop terms if necessary to avoid collinearity
        )
# `in_sample` creates features for the dates given in the `index` argument
X = dp.in_sample()

X.head()

In [ ]:
y = tunnel["NumVehicles"]

lrg = LinearRegression(fit_intercept=False).fit(X, y)
y_pred = pd.Series(lrg.predict(X),  index=X.index)

In [ ]:
ax = tunnel.plot(style=".", color="0.5", title="Tunnel Traffic - Linear Trend")
_ = y_pred.plot(ax=ax,  linewidth=3, label='Trend')

In [ ]:
# To make a forecast, we apply our model to "out of sample" features, Here's how we could make a 30-day forecast:
X = dp.out_of_sample(steps=30)
y_fore = pd.Series(lrg.predict(X), index=X.index)
y_fore.head()

In [ ]:
ax = tunnel['2005-05':].plot(title="Tunnel Traffic - Linear Trend Forecast")
ax = y_pred['2005-05':].plot(ax=ax, linewidth=3, label='Trend')
ax = y_fore.plot(ax=ax, linewidth=3, label='Trend Forecast', color='C3')

## 3. Seasonality
a time series exhibits seasonality whenever there is a regular, periodic change in the mean of the series. Seasonal changes generally follow the clock and calendar -- repetitions over a day, a week, or a year are common.

### Seasonal Plots and ( short snapshot )
A seasonal plot shows segments of the time series plotted against some common period, the period being the "season" you want to observe.
###  Seasonal Indicators
are binary features that represent seasonal differences in the level of a time series. Seasonal indicators are what you get if you treat a seasonal period as a categorical feature and apply one-hot encoding.

### Fourier Features and the Periodogram ( long snapshot )
The kind of feature we discuss now are better suited for long seasons over many observations where indicators would be impractical. Instead of creating a feature for each date, Fourier features try to capture the overall shape of the seasonal curve with just a few features.

#### Choosing Fourier features with the Periodogram
